<a href="https://colab.research.google.com/github/iskra3138/tft/blob/master/TFT_favorita_hm_no_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Jun  7 10:46:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from google.colab import drive

In [3]:
drive.mount('/Gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /Gdrive


In [4]:
cd '/Gdrive/My Drive/TFT'

/Gdrive/My Drive/TFT


In [5]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [6]:
cd tft

/Gdrive/My Drive/TFT/tft


# train

In [0]:
import argparse
import datetime as dte
import os

import data_formatters.base
import expt_settings.configs
import libs.hyperparam_opt
import libs.tft_model
import libs.utils as utils
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf

In [0]:
ExperimentConfig = expt_settings.configs.ExperimentConfig
HyperparamOptManager = libs.hyperparam_opt.HyperparamOptManager
ModelClass = libs.tft_model.TemporalFusionTransformer

In [0]:
# where $EXPT can be any of {volatility, electricity, traffic, favorita}, 
# and $OUTPUT_FOLDER denotes the root folder in which experiment outputs are saved.
EXPT='favorita'
OUTPUT_FOLDER='favorita_test'

In [10]:
# args for electricity testing from gpu
name, output_folder, use_tensorflow_with_gpu = EXPT, OUTPUT_FOLDER, True

print("Using output folder {}".format(output_folder))

config = ExperimentConfig(name, output_folder)
formatter = config.make_data_formatter()

Using output folder favorita_test


In [0]:
expt_name=name
use_gpu=use_tensorflow_with_gpu
model_folder=os.path.join(config.model_folder, "fixed")
data_csv_path=config.data_csv_path
data_formatter=formatter
#use_testing_mode=False

In [0]:
#num_repeats = 1

if not isinstance(data_formatter, data_formatters.base.GenericDataFormatter):
  raise ValueError(
      "Data formatters should inherit from" +
      "AbstractDataFormatter! Type={}".format(type(data_formatter)))

In [0]:
# Tensorflow setup
default_keras_session = tf.keras.backend.get_session()

In [14]:
if use_gpu:
  tf_config = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

else:
  tf_config = utils.get_default_tensorflow_config(tf_device="cpu")

print("*** Training from defined parameters for {} ***".format(expt_name))

Selecting GPU ID=0

*** Training from defined parameters for favorita ***


In [15]:
!pip install ipython-autotime

%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=9a9bc503a39fd4f9f20645dbfce235c6be1c70ebbdb89ee8a1e841bee13f95f1
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


# GCP Bucket에서 가져오기

In [16]:
from google.colab import auth
auth.authenticate_user()

time: 25.5 s


In [17]:
!gsutil cp gs://iskra3138_retail/temp/tft/favorita_train_fn.csv /content

Copying gs://iskra3138_retail/temp/tft/favorita_train_fn.csv...
/ [1 files][  3.5 GiB/  3.5 GiB]   49.9 MiB/s                                   
Operation completed over 1 objects/3.5 GiB.                                      
time: 1min 22s


In [18]:
!gsutil cp gs://iskra3138_retail/temp/tft/favorita_valid_fn.csv /content

Copying gs://iskra3138_retail/temp/tft/favorita_valid_fn.csv...
| [1 files][  1.6 GiB/  1.6 GiB]   45.5 MiB/s                                   
Operation completed over 1 objects/1.6 GiB.                                      
time: 40 s


In [19]:
!gsutil cp gs://iskra3138_retail/temp/tft/favorita_test_fn.csv /content

Copying gs://iskra3138_retail/temp/tft/favorita_test_fn.csv...
- [1 files][225.2 MiB/225.2 MiB]                                                
Operation completed over 1 objects/225.2 MiB.                                    
time: 5.12 s


In [20]:
raw_train = pd.read_csv('/content/favorita_train_fn.csv', index_col=0)
raw_train['date'] = pd.to_datetime(raw_train['date'])

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 55.9 s


In [21]:
raw_valid = pd.read_csv('/content/favorita_valid_fn.csv', index_col=0)
raw_valid['date'] = pd.to_datetime(raw_valid['date'])

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 22.8 s


In [22]:
raw_test = pd.read_csv('/content/favorita_test_fn.csv', index_col=0)
raw_test['date'] = pd.to_datetime(raw_test['date'])

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 3.34 s


In [23]:
raw_data = pd.concat([raw_train, raw_valid, raw_test])

time: 7.03 s


In [24]:
print (len(raw_train), len(raw_valid), len(raw_test))

22136420 9800160 1388640
time: 1.55 ms


In [25]:
data_formatter.set_scalers(raw_train, set_real=True)

Setting scalers with training data...
time: 1.23 s


In [26]:
data_formatter.set_scalers(raw_data, set_real=False)

Setting scalers with training data...
time: 2min 45s


In [27]:
import gc

time: 1.1 ms


In [28]:
del raw_data
gc.collect()

0

time: 739 ms


In [29]:
train = data_formatter.transform_inputs(raw_train)

time: 2min 2s


In [30]:
del raw_train
gc.collect()

0

time: 459 ms


In [31]:
valid = data_formatter.transform_inputs(raw_valid)

time: 53.3 s


In [32]:
del raw_valid
gc.collect()

0

time: 260 ms


In [33]:
test = data_formatter.transform_inputs(raw_test)

time: 7.14 s


In [34]:
del raw_test
gc.collect()

0

time: 112 ms


In [35]:
train_samples, valid_samples = data_formatter.get_num_samples_for_calibration()

time: 1.12 ms


In [36]:
# Sets up default params
fixed_params = data_formatter.get_experiment_params()
params = data_formatter.get_default_model_params()
params["model_folder"] = model_folder

time: 1.49 ms


In [37]:
# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                    fixed_params, model_folder)

*** Loading hyperparm manager ***
time: 8.9 ms


In [38]:
# Training -- one iteration only
print("*** Running calibration ***")
print("Params Selected:")
for k in params:
  print("{}: {}".format(k, params[k]))

*** Running calibration ***
Params Selected:
dropout_rate: 0.1
hidden_layer_size: 240
learning_rate: 0.001
minibatch_size: 128
max_gradient_norm: 100.0
num_heads: 4
stack_size: 1
model_folder: favorita_test/saved_models/favorita/fixed
time: 1.9 ms


In [39]:
params = opt_manager.get_next_parameters()

time: 9.35 ms


Next, we train a TFT model by subsampling the data in the training and validation sets.

In [40]:
tf.reset_default_graph()
with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    
    # Create a TFT model
    model = ModelClass(params, use_cudnn=True) # Run model on GPU using CuDNNLSTM cells

    # Sample data into minibatches for training
    if not model.training_data_cached():
        model.cache_batched_data(train, "train", num_samples=150000)
        model.cache_batched_data(valid, "valid", num_samples=50000)

    # Train and save model
    model.fit()
    model.save(model_folder)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Getting locations for 7_158842
Getting locations for 7_158875
Getting locations for 7_159156
Getting locations for 7_159242
Getting locations for 7_1609724
Getting locations for 7_1609729
Getting locations for 7_162066
Getting locations for 7_1625605
Getting locations for 7_164036
Getting locations for 7_164037
Getting locations for 7_164088
Getting locations for 7_1642397
Getting locations for 7_1642398
Getting locations for 7_1642399
Getting locations for 7_1642400
Getting locations for 7_1642401
Getting locations for 7_1642664
Getting locations for 7_164647
Getting locations for 7_165550
Getting locations for 7_165551
Getting locations for 7_165553
Getting locations for 7_165594
Getting locations for 7_165693
Getting locations for 7_165704
Getting locations for 7_165705
Getting locations for 7_165727
Getting locations for 7_1658994
Getting locations for 7_165988
Getting locations for 7_1660188
Getting locations for 7_1660191
Getting locations for 

To evaluate model performance, we reload the serialised model and compute P50 and P90 losses.

In [41]:
tf.reset_default_graph()
with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    
    # Create a new model & load weights
    model = ModelClass(params, use_cudnn=True)
    model.load(model_folder)
    
    # Make forecasts
    output_map = model.predict(test, return_targets=True)

    targets = data_formatter.format_predictions(output_map["targets"])

    # Format predictions
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
          col for col in data.columns
          if col not in {"forecast_time", "identifier"}
        ]]

    # Compute Losses
    p50_loss = utils.numpy_normalised_quantile_loss(
        extract_numerical_data(targets), extract_numerical_data(p50_forecast),
        0.5)
    p90_loss = utils.numpy_normalised_quantile_loss(
        extract_numerical_data(targets), extract_numerical_data(p90_forecast),
        0.9)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 -8.58971477e-03  1.63478553e-02 -1.54887913e-02  8.72579589e-03
 -2.99773831e-02 -3.27859297e-02  3.10031865e-02  2.17761155e-02
 -3.46983895e-02  8.38918518e-03  2.43969299e-02 -1.99929141e-02
 -1.61081962e-02 -1.18093928e-02 -6.89257286e-04  1.97489727e-02
 -2.67309472e-02  3.25755589e-02 -6.70605572e-03  1.19348271e-02
  2.80323755e-02 -1.87300388e-02 -5.53913694e-03  9.40094178e-04
  7.49600120e-03 -5.40248631e-03  6.88580470e-03  2.03691539e-03
 -7.10076001e-03  8.10795650e-03 -2.36975309e-02 -1.53569374e-02
 -1.35083860e-02  4.38892888e-03 -5.88515541e-03  8.10136553e-03
 -2.13398635e-02 -1.77606829e-02 -1.16165886e-02 -3.59106739e-03
 -6.15614327e-03 -1.26202097e-02  7.83878099e-03 -1.09991394e-02
  2.48865895e-02  3.19637125e-04 -5.00525767e-03 -5.10382233e-06
 -3.98261100e-02 -2.41566543e-03  2.74603646e-02 -1.29535999e-02
 -1.16693899e-02  3.20681222e-02  4.92940564e-03  8.72845389e-03
 -4.45457641e-03 -6.65272074e-03  2.97689382e-02  1.97

In [42]:
print("Normalised quantile losses: P50={}, P90={}".format(p50_loss.mean(), p90_loss.mean()))

Normalised quantile losses: P50=0.3318333979151835, P90=0.14713799454282495
time: 2.25 ms


## Interpretability Use Cases
The relationships learnt by the TFT can also be studied using the trained model, through:

1. Analyzing the variable selection weights to identify significant features for the prediction problem.
2. Visualizing distributions of self-attention weights to determine the presence of any persistent temporal relationships.

In the remainder of this section, we demonstrate two interpretability use cases to showcase the above.

### Generate Weights for Interpretability
First, we generate all necessary variable selection and attention weights required for analysis.

In [0]:
# Store outputs in maps
counts = 0
interpretability_weights = {k: None for k in ['decoder_self_attn', 
                                              'static_flags', 'historical_flags', 'future_flags']}

tf.reset_default_graph()
with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    
    # Create a new model & load weights
    model = ModelClass(params, 
                                    use_cudnn=True)
    model.load(model_folder)
    for identifier, sliced in test.groupby('traj_id'):
        
        print("Getting attention weights for {}".format(identifier))
        weights = model.get_attention(sliced)
        
        for k in interpretability_weights:
            w = weights[k]
            
            # Average attentin across heads if necessary
            if k == 'decoder_self_attn':
                w = w.mean(axis=0)
            
                # Store a single matrix for weights to reduce memory footprint
                batch_size, _, _ = w.shape                 
                counts += batch_size
            
            if interpretability_weights[k] is None:
                interpretability_weights[k] = w.sum(axis=0)
            else:
                interpretability_weights[k] += w.sum(axis=0)

interpretability_weight = {k: interpretability_weights[k]/counts for k in interpretability_weights}

print('Done.')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
   0.00594103]
 [ 0.00378726  0.04894188 -0.03154607 ... -0.09501971  0.12360397
  -0.00322568]
 ...
 [-0.12562133 -0.04412106 -0.02893431 ... -0.07334027  0.24671729
   0.07790156]
 [ 0.17260598  0.06131192 -0.09956042 ... -0.01664287 -0.01053553
   0.05038585]
 [-0.03069553  0.0087095  -0.0712318  ... -0.04053007 -0.13803358
  -0.00579988]]
tensor_name:  TemporalFusionTransformer/time_distributed_67/bias
[ 0.01946679 -0.01898478 -0.08332723 -0.00681202 -0.04756457 -0.15155014
  0.11448463 -0.12071174  0.00182302 -0.04077926 -0.13143224  0.00491782
 -0.00159931  0.00433443 -0.00459724  0.01303445  0.063203    0.00927456
 -0.0708049  -0.01666487 -0.07344899  0.02179372 -0.05540891 -0.01600887
  0.01985352  0.13938542  0.07808566 -0.01089944 -0.04475059 -0.05810319
  0.08994815  0.07801502  0.02966555 -0.0437106   0.01127718 -0.03881334
 -0.00908395 -0.01196778  0.03641704  0.02141306 -0.05723217 -0.1377411
 -0.0321286   0.01290016  0.13775171 -0.1145

### Use Case 1: Analyzing Variable Importance 

Next, we analyze the distribution of variable selection weights on the input layer -- using this to quantify the relative importance of a given feature for the prediction problem in general. This is split into variable importance for static covariates, time-varying historical inputs variables and known future inputs as shown below.

In [0]:
import numpy as np
def get_range(static_gate, axis=None):
    """Returns the mean, 10th, 50th and 90th percentile of variable importance weights."""
    return {'Mean': static_gate.mean(axis=axis), 
               '10%': np.quantile(static_gate, 0.1, axis=axis),
               '50%': np.quantile(static_gate, 0.5, axis=axis),
               '90%': np.quantile(static_gate, 0.9, axis=axis)}

#### Static Variable Importance

In [0]:
def flatten(x):
    static_attn = x
    static_attn = static_attn.reshape([-1, static_attn.shape[-1]])
    return static_attn

static_attn = flatten(interpretability_weights['static_flags'])
m = get_range(static_attn, axis=0)
pd.DataFrame({k: pd.Series(m[k], index=['ID']) for k in m})

#### Temporal Variable Importance -- Past Inputs

In [0]:
x = flatten(interpretability_weights['historical_flags'])
m = get_range(x, axis=0)
pd.DataFrame({k: pd.Series(m[k], index=['Hour of Day', 'Day of Week', 'Time Index', 'Target']) for k in m})

#### Temporal Variable Importance -- Future Inputs

In [0]:
x = flatten(interpretability_weights['future_flags'])
m = get_range(x, axis=0)
pd.DataFrame({k: pd.Series(m[k], index=['Hour of Day', 'Day of Week', 'Time Index']) for k in m})

### Use Case 2: Visualizing Persistent Temporal Patterns
Lastly, we analyse the distribution of self-attention weights across various horizons to see if any persistent temporal patterns exist within the dataset. This allows us to identify any seasonal patterns or lagged relationships in the dataset, based on which past time steps are consistently important for predictions at a given horizon. 

We visualize this using the average attention pattern for various prediction horizons, as shown below:

#### Mean Attention Weights for Various Prediction Horizons

In [0]:
# Plotting libraries & Functions
import plotly.offline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot
import plotly.graph_objs as go
import cufflinks as cf
from IPython.display import HTML

# Loads plotly charts
def iplot(fig, s='plot.html'):
    filename = os.path.join(output_folder, s)
    plotly.offline.plot(fig, filename=filename, auto_open=False)
    return HTML(filename)    

def plotly_chart(df, title=None, kind='scatter', x_label=None, y_label=None, secondary_y=None, fill=None,
                shape=None, subplots=False):
    
    fig = df.iplot(asFigure=True, title=title, kind=kind, xTitle=x_label, yTitle=y_label, secondary_y=secondary_y,
                  fill=fill, subplots=subplots, shape=shape)

    return iplot(fig)

In [0]:
self_attn = interpretability_weights['decoder_self_attn']

means = pd.DataFrame({"horizon={}".format(k): self_attn[model.num_encoder_steps+k-1, :] 
                      for k in [1, 5, 10, 15, 20]})
means.index -= model.num_encoder_steps

plotly_chart(means,
             x_label="Positiion Index (n)",
             y_label="Mean Attention Weight",
             title="Average Attention Pattern at Various Prediction Horizons")